In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', None)  # Show all columns in DataFrame output
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from pyod.models.abod import ABOD
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('C:\\Users\\Jagdish\\imdb_4.csv')
df.drop(columns = ['canHaveEpisodes'], inplace = True)
df

,startYear,runtimeMinutes,totalCredits,numRegions,ratingCount,castNumber,companiesNumber,writerCredits,directorsCredits,totalNominations,totalMedia,totalReviews,Asia,Africa,Europe,North America,South America,Oceania,Continent Unknown,genre1,genre2,genre3,movie,short,tvEpisode,tvMiniSeries,tvMovie,tvSeries,tvShort,tvSpecial,video,videoGame,rating_bin
0,1894,1,4,7,2092,1,3,0,1,0,11,19,1,0,5,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0,1
1,1892,12,2,6,183,0,0,0,1,0,5,1,1,0,5,0,0,0,0,17855,16787,0,0,1,0,0,0,0,0,0,0,0,1
2,1894,1,1,5,195,0,1,0,1,0,5,0,0,0,4,1,0,0,0,16787,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1894,1,4,6,2238,1,6,0,1,1,9,22,1,0,4,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0,1
4,1896,1,11,21,13115,6,5,0,2,0,33,82,2,0,14,2,1,0,2,16787,16581,0,0,1,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,1993,96,11,1,11,0,2,1,1,0,1,0,0,0,0,0,1,0,0,16581,0,0,1,0,0,0,0,0,0,0,0,0,3
149527,2019,14,52,1,15,6,3,1,1,3,10,1,0,0,1,0,0,0,0,51745,16787,0,0,1,0,0,0,0,0,0,0,0,2
149528,2019,29,32,0,12,1,0,0,0,0,5,0,0,0,0,0,0,0,0,5726,0,0,0,0,1,0,0,0,0,0,0,0,1
149529,2011,10,15,0,10,7,0,3,1,0,1,0,0,0,0,0,0,0,0,47408,17855,17763,0,0,1,0,0,0,0,0,0,0,3


# 1% outlier

In [ ]:
%%time
# Select only numerical columns from the dataframe and ensure they are numeric
numerical_df = df.select_dtypes(include=['number']).apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values (if any) after conversion
numerical_df = numerical_df.dropna()

# normalizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numerical_df)

# Initialize the ABOD model
abod = ABOD(n_neighbors=50, method="fast", contamination=0.01)

# Fit the model to the numerical data
abod.fit(X_scaled)

# Predict the outliers based on decision scores
outlier_predictions = abod.decision_function(numerical_df)

CPU times: total: 1h 14min 33s
Wall time: 1h 17min 24s


In [7]:
# Sort the decision scores to identify the top 1% as outliers
threshold = sorted(outlier_predictions)[int(len(outlier_predictions) * 0.99)]

# Label outliers as -1 and inliers as 1 based on the threshold
df['ABOD_Score'] = [-1 if score >= threshold else 1 for score in outlier_predictions]

# Display the counts of inliers (1) and outliers (-1)
print(df['ABOD_Score'].value_counts())

ABOD_Score
 1    148035
-1      1496
Name: count, dtype: int64


In [8]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['ABOD_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'ABOD_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.41
Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.24      0.30      2523
           1       0.35      0.15      0.21      3891
           2       0.37      0.37      0.37      7323
           3       0.43      0.71      0.53      9720
           4       0.39      0.22      0.28      5057
           5       0.59      0.13      0.21      1093

    accuracy                           0.41     29607
   macro avg       0.42      0.30      0.32     29607
weighted avg       0.40      0.41      0.38     29607

CPU times: total: 2min 24s
Wall time: 2min 35s


In [10]:
%%time
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3953119194785017

Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.25      0.29      2523
           1       0.34      0.16      0.22      3891
           2       0.36      0.35      0.35      7323
           3       0.42      0.76      0.54      9720
           4       0.41      0.09      0.14      5057
           5       0.36      0.06      0.10      1093

    accuracy                           0.40     29607
   macro avg       0.37      0.28      0.28     29607
weighted avg       0.39      0.40      0.35     29607

CPU times: total: 312 ms
Wall time: 432 ms


# 5% outlier

In [ ]:
%%time
# Select only numerical columns from the dataframe and ensure they are numeric
numerical_df = df.select_dtypes(include=['number']).apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values (if any) after conversion
numerical_df = numerical_df.dropna()

# normalizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numerical_df)

# Initialize the ABOD model
abod = ABOD(n_neighbors=50, method="fast", contamination=0.05)

# Fit the model to the numerical data
abod.fit(X_scaled)

# Predict the outliers based on decision scores
outlier_predictions = abod.decision_function(numerical_df)

# Sort the decision scores to identify the top 1% as outliers
threshold = sorted(outlier_predictions)[int(len(outlier_predictions) * 0.95)]

# Label outliers as -1 and inliers as 1 based on the threshold
df['ABOD_Score'] = [-1 if score >= threshold else 1 for score in outlier_predictions]

# Display the counts of inliers (1) and outliers (-1)
print(df['ABOD_Score'].value_counts())

ABOD_Score
 1    148035
-1      1496
Name: count, dtype: int64
CPU times: total: 57min 9s
Wall time: 55min 52s


In [8]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['ABOD_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'ABOD_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.41
Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.24      0.30      2523
           1       0.35      0.15      0.21      3891
           2       0.37      0.37      0.37      7323
           3       0.43      0.71      0.53      9720
           4       0.39      0.22      0.28      5057
           5       0.59      0.13      0.21      1093

    accuracy                           0.41     29607
   macro avg       0.42      0.30      0.32     29607
weighted avg       0.40      0.41      0.38     29607

CPU times: total: 1min 50s
Wall time: 1min 50s


In [10]:
%%time
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3953119194785017

Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.25      0.29      2523
           1       0.34      0.16      0.22      3891
           2       0.36      0.35      0.35      7323
           3       0.42      0.76      0.54      9720
           4       0.41      0.09      0.14      5057
           5       0.36      0.06      0.10      1093

    accuracy                           0.40     29607
   macro avg       0.37      0.28      0.28     29607
weighted avg       0.39      0.40      0.35     29607

CPU times: total: 250 ms
Wall time: 244 ms


# 10% outlier

In [ ]:
%%time
# Select only numerical columns from the dataframe and ensure they are numeric
numerical_df = df.select_dtypes(include=['number']).apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values (if any) after conversion
numerical_df = numerical_df.dropna()

# normalizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numerical_df)

# Initialize the ABOD model
abod = ABOD(n_neighbors=50, method="fast", contamination=0.1)

# Fit the model to the numerical data
abod.fit(X_scaled)

# Predict the outliers based on decision scores
outlier_predictions = abod.decision_function(numerical_df)

# Sort the decision scores to identify the top 1% as outliers
threshold = sorted(outlier_predictions)[int(len(outlier_predictions) * 0.90)]

# Label outliers as -1 and inliers as 1 based on the threshold
df['ABOD_Score'] = [-1 if score >= threshold else 1 for score in outlier_predictions]

# Display the counts of inliers (1) and outliers (-1)
print(df['ABOD_Score'].value_counts())

ABOD_Score
 1    148035
-1      1496
Name: count, dtype: int64
CPU times: total: 15min 7s
Wall time: 13min 47s


In [4]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['ABOD_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'ABOD_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.41
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.24      0.31      2490
           1       0.34      0.15      0.21      3851
           2       0.37      0.38      0.38      7314
           3       0.44      0.71      0.54      9945
           4       0.39      0.21      0.27      4972
           5       0.58      0.14      0.22      1035

    accuracy                           0.41     29607
   macro avg       0.43      0.30      0.32     29607
weighted avg       0.41      0.41      0.38     29607

CPU times: total: 1min 50s
Wall time: 1min 50s


In [6]:
%%time
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.39787887999459587

Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.24      0.30      2490
           1       0.33      0.19      0.24      3851
           2       0.36      0.33      0.34      7314
           3       0.42      0.77      0.54      9945
           4       0.40      0.08      0.13      4972
           5       0.27      0.04      0.07      1035

    accuracy                           0.40     29607
   macro avg       0.36      0.27      0.27     29607
weighted avg       0.38      0.40      0.35     29607

CPU times: total: 188 ms
Wall time: 228 ms
